In [1]:
import psycopg2
import pandas as pd
import pandas.io.sql as pd_sql
from pandas import Series, DataFrame, Panel
import psycopg2
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
from pylab import *
import matplotlib.dates as mdates
import datetime as date
import scipy.optimize
import scipy.stats

In [14]:
conn_pg = psycopg2.connect("dbname='goldrush' user='pkelly' host=192.168.19.206 password='pkellypkelly'");
fig, ax = plt.subplots()
fig.set_size_inches(11,8.5)
#ax = fig.add_subplot(111)
pp = PdfPages('StackedWLs_'+date.datetime.now().strftime("%m_%d_%Y")+'.pdf')

In [3]:
## Create Layers File##
layerz = { 'layer':[1,	2,	3,	4,	5,	6,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	24], 'bottom_elev':[8050,	7250,	6600,	6050,	5850,	5650,	5450,	5250,	5050,	4850,	4650,	4450,	4250,	4050,	3850,	3650,	3450,	3250,	3050,	2850,	2650,	2450,	2250,	2050]}
layers = pd.DataFrame(layerz, columns=['layer','bottom_elev'])

## Get list of hole names and measure locations##
hole_names='''SELECT hole_name,measure_loc,rid FROM well_info ORDER BY hole_name;'''
All_locs = pd_sql.read_frame(hole_names,conn_pg)
#print Holes
distinct_names='''SELECT DISTINCT(hole_name) FROM well_info ORDER BY hole_name;'''
holes= pd_sql.read_frame(distinct_names,conn_pg)
#Select out the measurement locations
measure_mps = '''SELECT rid,mon_mp FROM transducer_fmn WHERE rid IS NOT NULL ORDER BY rid;'''
Mon_locs = pd_sql.read_frame(measure_mps,conn_pg)

In [4]:
##Query out all of the WL data: only need to do this once ##
wldp='''SELECT * FROM water_level_elev_daily_poly ORDER BY piezo_id;'''
WLs = pd_sql.read_frame(wldp,conn_pg)
WLpivot = WLs.pivot(index='measured_date_day', columns='piezo_id', values='water_level_elev_daily_poly')
WLpivot.to_excel('gw_pivot.xlsx')

In [15]:
Cnt_WL=0
first = True

for hole in holes.ix[:,0]:

    Cnt_WL+=1
    piezo_list = All_locs.rid[All_locs.hole_name == hole]
    time_series = WLs[WLs['piezo_id'].isin(piezo_list)] 
    measurelocs = Mon_locs[Mon_locs['rid'].isin(piezo_list)]
    ts_piv = time_series.pivot(index='measured_date_day', columns='piezo_id', values='water_level_elev_daily_poly')
    
    ##Plot WLs##
    ax = ts_piv.plot()
    
    try:
        date_min,date_max = min(time_series.icol(1)), max(time_series.icol(1))
        y_min,y_max = min(time_series.icol(2)), max(time_series.icol(2))
        layer_min,layer_max = min(measurelocs.icol(1)), max(measurelocs.icol(1))
        plot_layers = pd.concat([time_series.icol(2),measurelocs.icol(1)])
        plot_min,plot_max = min(plot_layers),max(plot_layers)
    
    except:
        continue
        
    for measure in measurelocs['mon_mp']:
        ax.plot(date_min,measure, 'o')
    
    layer_cnt = 0 
    for layer in layers.icol(1):
        layer_cnt+=1
        if plot_min < layer and layer < plot_max:
            ax.plot((date_min, date_max),(layer,layer),'k--')
            #time_series = layers[layers['bottom_elev'].isin(piezo_list)] 
            ax.annotate(layer_cnt, (mdates.date2num(date_min) + 20,layer), xytext=(15, 15), textcoords='offset points', arrowprops=dict(arrowstyle='-|>'))
        #else 

        
    #plt.show()
    ax.ticklabel_format(axis='y',useOffset=False)
    plt.xticks(rotation=70)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d-%Y'))
    ax.xaxis.set_major_locator(mdates.ticker.MaxNLocator(nbins=10))
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Water Level Elevation')
    
    fig.autofmt_xdate()
    plt.tight_layout()
    pp.savefig()
    print(str(Cnt_WL+1)+' of '+str(len(holes)))
    plt.clf()
pp.close()

In [12]:
min(plot_layers)

TypeError: can't compare datetime.date to float

In [32]:
print measurelocs

            rid       mon_mp
0  GRC-0010D P1  4927.709088
1  GRC-0010D P2  5466.742784
2  GRC-0010D P3  6004.666942


In [22]:
mdates.date2num(date_min)

735223.0

In [47]:
print measurelocs

Empty DataFrame
Columns: [rid, mon_mp]
Index: []


In [52]:
len(measurelocs.icol(1))

0